In [ ]:
import os
import requests
import json
from typing import List
from dotenv import load_dotenv
from bs4 import BeautifulSoup
from IPython.display import Markdown, display, update_display
from openai import OpenAI
from google import genai
from google.genai.types import *
from typing import List
import gradio as gr

In [ ]:
load_dotenv()

API_KEY = os.getenv(key="GEMINI_API_KEY")

if API_KEY:
    print("API_KEY found.")
else:
    raise "API Key not found."

gemini = genai.Client(api_key=API_KEY)

In [ ]:
'''
Contents Example
[
    {"role": "user", "parts": [{"text": "What is your name? My Name is real x"}]},
    {
        "role": "assistant",
        "parts": [{"text": "I am a large language model, trained by Google."}],
    },
    {"role": "user", "parts": [{"text": "Tell My Name"}]},
]
'''


def generate_content(contents, system_prompt):
    response = gemini.models.generate_content(
        model="gemini-2.0-flash",
        contents=contents,
        config={"system_instruction": {"text": system_prompt}},
    )

    return response.candidates[0].content.parts[0].text


def generate_content_stream(contents, system_prompt):
    response = gemini.models.generate_content_stream(
        model="gemini-2.0-flash",
        contents=contents,
        config={"system_instruction": {"text": system_prompt}},
    )

    for chunk in response:
        yield chunk.candidates[0].content.parts[0].text or ""


def getChatItem(role, prompt):
    # Content(role=role, parts=[Part(text=prompt)])  # This also works
    return {"role": role, "parts": [{"text": prompt}]}

In [ ]:
def chat(user_prompt, history):
    system_prompt = "You are a helpful assistant in a clothes store. You should try to gently encourage \
        the customer to try items that are on sale. Hats are 60% off, and most other items are 50% off. \
        For example, if the customer says 'I'm looking to buy a hat', \
        you could reply something like, 'Wonderful - we have lots of hats - including several that are part of our sales event.'\
        Encourage the customer to buy hats if they are unsure what to get."

    contents = []
    for item in history:
        contents.append(getChatItem(item['role'], item['content']))
    contents.append(getChatItem("user", user_prompt))

    return generate_content(contents, system_prompt)

In [ ]:
gr.ChatInterface(fn=chat, type="messages").launch()